# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession

session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	8433 candidates
Dev set:	920 candidates
Test set:	4683 candidates


# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

**Note: Again, training for more epochs than below will greatly improve performance- try it out!**

In [3]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, split=0)

In [4]:
train_marginals.shape

(8433,)

In [5]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

In [27]:
train_marginals.shape

(8433,)

In [6]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':              0.01,
    'embedding_dim':   100,
    'hidden_dim':      100,
    'n_epochs':        20,
    'dropout':         0.5,
    'rebalance':       0.25,
    'print_freq':      5,
    'seed':            1701
}

lstm = LSTM(n_threads=None)
lstm.train(train, train_marginals, X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=3587  #epochs=20  batch size=64


/home/cperreault/anaconda3/envs/snorkel-extraction/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[LSTM] Epoch 1 (16.71s)	Average loss=0.686261	Dev F1=51.49
[LSTM] Epoch 6 (99.85s)	Average loss=0.663766	Dev F1=52.09
[LSTM] Epoch 11 (180.72s)	Average loss=0.659170	Dev F1=52.47
[LSTM] Epoch 16 (260.90s)	Average loss=0.658514	Dev F1=52.50
[LSTM] Epoch 20 (326.44s)	Average loss=0.656779	Dev F1=53.18
[LSTM] Model saved as <LSTM>
[LSTM] Training done (327.47s)
[LSTM] Loaded model <LSTM>


### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

In [7]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

AnnotatorLabels created: 0


<4683x1 sparse matrix of type '<class 'numpy.int64'>'
	with 4683 stored elements in Compressed Sparse Row format>

In [8]:
lstm.score(test, L_gold_test)

(0.37826086956521737, 0.8619550858652576, 0.52578565672844468)

In [9]:
hand = np.loadtxt('handlabeled_predictions.txt')

In [10]:
out = lstm.marginals(test)
pred = np.zeros(out.shape)
pred[out >=.5] = 1

In [11]:
c = 0
for e,x in enumerate(pred):
    if x != hand[e]:
        c += 1

In [12]:
c

2483

In [13]:
len(pred)

4683

In [14]:
true = L_gold_test.toarray().reshape(4683,)
handTrue = np.loadtxt('test.txt')
true[true == -1] = 0

In [15]:
for e,x in enumerate(true):
    if x != handTrue[e]:
        print(e,"no match")
        break

In [16]:
true

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
handTrue

array([ 0.,  1.,  0., ...,  0.,  0.,  0.])

In [18]:
lstm.save(model_name='weak_supervision')

[LSTM] Model saved as <weak_supervision>


In [21]:
features = lstm.feature_outputs(test,100)

In [22]:
features.shape

torch.Size([46830])

In [25]:
features = features.reshape(4683,10).detach().numpy()
cov = np.matmul(features.transpose(),features)

In [26]:
np.linalg.svd(cov)

(array([[-0.03753358,  0.2017083 ,  0.29510844,  0.37143236,  0.09484822,
          0.25345919, -0.10347617,  0.08641696, -0.78299898,  0.16838549],
        [-0.0072735 , -0.37860715,  0.31895134,  0.30586937,  0.01223327,
          0.24773405,  0.74877024, -0.07050744,  0.11132964, -0.14755824],
        [-0.97038037,  0.0160363 , -0.01084041, -0.01285646, -0.04549643,
         -0.07253069, -0.00197182,  0.03527078, -0.0316091 , -0.21964598],
        [-0.02340209,  0.3760922 ,  0.72985381, -0.44445309,  0.29165339,
         -0.04730985,  0.05445272,  0.00917324,  0.18686424,  0.05014446],
        [ 0.0331578 , -0.33265111,  0.18049695, -0.10193478,  0.00814971,
         -0.42463708, -0.11076105, -0.74365056, -0.29540676, -0.11109271],
        [-0.02671201, -0.22555141, -0.0285029 ,  0.33878416,  0.72534454,
         -0.4553903 , -0.06846263,  0.26920882,  0.09669558,  0.11318279],
        [ 0.21310924, -0.03591174,  0.01963841, -0.12391026,  0.12060343,
          0.01353883, -0.1012408